#My ELT Pipeline project
Phase 1: Python Extraction (EL)

In [ ]:
import requests
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector

# API Endpoints
endpoints = {
    "products": "https://api.escuelajs.co/api/v1/products",
    "users": "https://api.escuelajs.co/api/v1/users",
    "categories": "https://api.escuelajs.co/api/v1/categories"
     # Carts API is often empty/unstable in this set, using categories for logic
}

def get_data(endpoint):
    response = requests.get(endpoint)
    return response.json()

# Snowflake Connection
ctx = snowflake.connector.connect(
    user='BABAK6178',
    password='Bonyasophia202#',
    account='LQKWSCJ-MFC23023',
    warehouse='COMPUTE_WH',
    database='ELT_DB',
    schema='EXTERNAL_DATA'
)

for table_name, url in endpoints.items():
    data = get_data(url)
    df = pd.DataFrame(data)
    
    # Flattening nested JSON if necessary (Example: Category in Products)
    if table_name == 'products':
        df['category_id'] = df['category'].apply(lambda x: x['id'] if isinstance(x, dict) else None)
        df = df.drop(columns=['category', 'images']) # Cleaning for simplicity

    # Write to Snowflake
    success, nchunks, nrows, _ = write_pandas(ctx, df, table_name.upper(), auto_create_table=True)
    print(f"Uploaded {nrows} rows to {table_name.upper()}")

ctx.close()

Uploaded 53 rows to PRODUCTS
Uploaded 3 rows to USERS
Uploaded 5 rows to CATEGORIES


In [4]:
# Example of what might be going wrong:
loading = True
profile = None

def load_data():
    global loading, profile
    try:
        profile = fetch_user()
    finally:
        loading = False # If fetch_user returns None, you get the error!

In [6]:
conn_params = {
"user": "BABAK6178", 
    "password": "Bonyasophia202#",
    "account": "LQKWSCJ-MFC23023", # Remove .snowflakecomputing.com
    "warehouse": "COMPUTE_WH",
    "database": "RAW_DB",
    "schema": "EXTERNAL_DATA",
    "role": "ACCOUNTADMIN"
}

In [10]:
import snowflake.connector

# Replace with your actual credentials
MY_USER = 'BABAK6178' 
MY_PASSWORD = 'Bonyasophia202#'
MY_ACCOUNT = 'LQKWSCJ-MFC23023' 

try:
    ctx = snowflake.connector.connect(
        user=MY_USER.upper(),
        password=MY_PASSWORD,
        account=MY_ACCOUNT
    )
    cs = ctx.cursor()
    cs.execute("SELECT current_version(), current_role()")
    data = cs.fetchone()
    print("\n✅ CONNECTION SUCCESSFUL!")
    print(f"Snowflake Version: {data[0]}")
    print(f"Current Role:      {data[1]}")
    print("------------------------------\n")
    ctx.close()
except Exception as e:
    print(f"\n❌ CONNECTION FAILED:\n{e}\n")


✅ CONNECTION SUCCESSFUL!
Snowflake Version: 10.0.0
Current Role:      ACCOUNTADMIN
------------------------------

